In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torchvision import models
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time
import copy
from tqdm import tqdm

In [2]:
!pip install --upgrade -q kaggle

!mkdir /root/.kaggle
import json
token = {
    "username": "zahidhussain909",
    "key": "39a06efd89d0f2a699143b8d3d62b216"
}

with open('/root/.kaggle/kaggle.json', 'w') as config_file:
    json.dump(token, config_file)
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d mohamedberrimi/oct-images-balanced-version

import zipfile
zipref=zipfile.ZipFile("/content/oct-images-balanced-version.zip",'r')
zipref.extractall()
zipref.close()

!rm -rf /content/oct-images-balanced-version.zip


100% 4.15G/4.15G [00:29<00:00, 152MB/s]
100% 4.15G/4.15G [00:30<00:00, 149MB/s]


### Load the dataset

In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, ds_path, transforms=None, target_transform=None):
        self.ds_path = ds_path
        self.transforms = transforms
        self.target_transform = target_transform
        self.labels = os.listdir(ds_path)
        img_paths = []
        
        for i in self.labels:
            base_path = os.path.join(ds_path, i)
            imgs = os.listdir(base_path)
            for img in imgs:
                img_path = base_path + '/' + img
                img_paths.append(img_path)
                
        self.img_paths = img_paths
        self.label_map = {'CNV':0, 'DME':1, 'DRUSEN':2, 'NORMAL':3,}

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = read_image(img_path)
        label = img_path.split('/')[-2]
        image = image.squeeze()
        image = image.repeat(3, 1, 1)
        image = image / 255.0
        if self.transforms:
            image = self.transforms(image)
        if self.target_transform:
            label = self.target_transform(label)
            
        image = T.Resize((224, 224))(image)
        
        return image, self.label_map[label]

In [4]:
# Load training dataset
train_path = '/content/OCT_Balanced_Version/Balanced'
train_data = CustomImageDataset(train_path)

# Load validation dataset
val_path = '/content/test/test'
val_data = CustomImageDataset(val_path)

# Load test dataset
test_path = '/content/test/test'
test_data = CustomImageDataset(test_path)

In [5]:
from torch.utils.data import DataLoader

train_len = len(train_data)
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

val_len = len(val_data)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)

test_len = len(test_data)
test_dataloader = DataLoader(test_data, batch_size=8)

data_loaders = {'train':train_dataloader, 'val':val_dataloader}

Feature batch shape: torch.Size([64, 3, 224, 224])
Labels batch shape: torch.Size([64])


Label: 2


In [6]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img.permute(1, 2, 0))
plt.show()
print(f"Label: {label}")

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

Parameters:



out_channels (int) – Number of channels produced by the convolution

kernel_size (int or tuple) – Size of the convolving kernel

stride (int or tuple, optional) – Stride of the convolution. Default: 1

padding (int, tuple or str, optional) – Padding added to all four sides of the input. Default: 0

padding_mode (str, optional) – 'zeros', 'reflect', 'replicate' or 'circular'. Default: 'zeros'

dilation (int or tuple, optional) – Spacing between kernel elements. Default: 1

groups (int, optional) – Number of blocked connections from input channels to output channels. Default: 1

bias (bool, optional) – If True, adds a learnable bias to the output. Default: True

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [8]:
class Cifar10CnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 70, kernel_size=3),
            nn.ReLU(),

            nn.BatchNormalization(),

            nn.MaxPool2d(3, 3, padding='valid'), # output: 64 x 16 x 16

            nn.Conv2d(70, 50, kernel_size=3),
            nn.ReLU(),

            nn.BatchNormalization(),

            nn.MaxPool2d(3, 3, padding='valid'), # output: 64 x 16 x 16

            nn.Conv2d(50, 20, kernel_size=3),
            nn.ReLU(),

            nn.BatchNormalization(),

            nn.MaxPool2d(3, 3, padding='valid'), # output: 64 x 16 x 16

            nn.Flatten(), 

            nn.Linear(256*4*4, 4),
            nn.Softmax()
        
    def forward(self, xb):
        return self.network(xb)

NameError: ignored

In [ ]:
model = Cifar10CnnModel()
model

In [ ]:
#using batchnormalization after each conv2d    -tick
custom4 = tf.keras.Sequential([
                              tf.keras.layers.Conv2D(70,3,activation='relu',input_shape=(224,224,3)),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.MaxPool2D(pool_size=3,padding='valid'),
                              tf.keras.layers.Conv2D(50,3,activation='relu'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.MaxPool2D(),
                              tf.keras.layers.Conv2D(20,3,activation='relu'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.MaxPool2D(),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(4,activation='softmax')
])


custom4.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
               metrics=[tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.PrecisionAtRecall(0.5),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.SpecificityAtSensitivity(0.5),'accuracy'])

history1_9=custom4.fit(train_dataset,epochs=1,steps_per_epoch=len(train_dataset),validation_data=val_dataset,validation_steps=len(val_dataset),
                    callbacks = callbacks)

In [ ]:
model = Cifar10CnnModel()
model

## Create Training function

In [9]:
def train_model(model, criterion, optimizer, num_epochs=3):
    since = time.time()
    
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)
        for mode, data_loader in data_loaders.items():
            if mode == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_correct = 0

            # Iterate over batches
            for inputs, labels in tqdm(data_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                with torch.set_grad_enabled(mode == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if mode == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_correct += torch.sum(preds==labels.data)
            
            data_len = train_len
            
            if mode == 'val':
                data_len = val_len
            
            epoch_loss = running_loss / data_len
            epoch_acc = running_correct.double() / data_len

            print('{} loss: {:4f} acc: {:4f}'.format(mode, epoch_loss, epoch_acc))
        
    

## Defining model

In [10]:
train_len, val_len, test_len

(32064, 968, 968)

In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
train_model(model, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------


100%|██████████| 1305/1305 [16:47<00:00,  1.30it/s]


train loss: 0.197729 acc: 0.934407


100%|██████████| 1/1 [00:00<00:00,  3.23it/s]


val loss: 0.161058 acc: 0.906250
Epoch 2/10
----------


100%|██████████| 1305/1305 [10:04<00:00,  2.16it/s]


train loss: 0.133178 acc: 0.954566


100%|██████████| 1/1 [00:00<00:00,  5.77it/s]


val loss: 0.075461 acc: 1.000000
Epoch 3/10
----------


100%|██████████| 1305/1305 [09:54<00:00,  2.20it/s]


train loss: 0.109112 acc: 0.962915


100%|██████████| 1/1 [00:00<00:00,  6.12it/s]


val loss: 0.045358 acc: 0.968750
Epoch 4/10
----------


100%|██████████| 1305/1305 [09:38<00:00,  2.25it/s]


train loss: 0.094107 acc: 0.967539


100%|██████████| 1/1 [00:00<00:00,  6.50it/s]


val loss: 0.027706 acc: 1.000000
Epoch 5/10
----------


100%|██████████| 1305/1305 [09:57<00:00,  2.18it/s]


train loss: 0.076797 acc: 0.973157


100%|██████████| 1/1 [00:00<00:00,  6.22it/s]


val loss: 0.043634 acc: 0.968750
Epoch 6/10
----------


100%|██████████| 1305/1305 [09:57<00:00,  2.19it/s]


train loss: 0.063644 acc: 0.977972


100%|██████████| 1/1 [00:00<00:00,  5.67it/s]


val loss: 0.028500 acc: 1.000000
Epoch 7/10
----------


100%|██████████| 1305/1305 [09:52<00:00,  2.20it/s]


train loss: 0.053975 acc: 0.980942


100%|██████████| 1/1 [00:00<00:00,  6.19it/s]


val loss: 0.010490 acc: 1.000000
Epoch 8/10
----------


100%|██████████| 1305/1305 [09:53<00:00,  2.20it/s]


train loss: 0.043672 acc: 0.985111


100%|██████████| 1/1 [00:00<00:00,  6.09it/s]


val loss: 0.005956 acc: 1.000000
Epoch 9/10
----------


100%|██████████| 1305/1305 [10:04<00:00,  2.16it/s]


train loss: 0.037629 acc: 0.987147


100%|██████████| 1/1 [00:00<00:00,  6.23it/s]


val loss: 0.014543 acc: 1.000000
Epoch 10/10
----------


100%|██████████| 1305/1305 [09:53<00:00,  2.20it/s]


train loss: 0.034859 acc: 0.987279


100%|██████████| 1/1 [00:00<00:00,  6.35it/s]

val loss: 0.002767 acc: 1.000000


In [ ]:
running_loss = 0.0
running_correct = 0

# Iterate over batches
for inputs, labels in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

    running_loss += loss.item() * inputs.size(0)
    running_correct += torch.sum(preds==labels.data)
    
epoch_loss = running_loss / test_len
epoch_acc = running_correct.double() / test_len

print('{} loss: {:4f} acc: {:4f}'.format('test', epoch_loss, epoch_acc))

100%|██████████| 121/121 [00:09<00:00, 12.92it/s]

test loss: 0.012862 acc: 0.996901
